# **RC4**

Rivest Cipher 4 es un algoritmo de cifrado de flujo de clave simétrica diseñado por Ron Rivest en el 87. Era muy famoso por ser muy simple y fácil a nivel de software por lo que encontró aplicación en una amplia variedad de criptosistemas como SSL/ TLS, WEP y WPA2. Sin embargo, estas ventajas tienen costos en términos de que su seguridad se ve reducida, por lo que rápidamente se encontraron varias vulnerabilidades, especialmente cuando el comienzo del flujo de clave de salida no se descarta, o cuando se utilizan claves no aleatorias o relacionadas. Esto llevó al IETF a prohibir el uso de RC4 en TLS(protocolo utilizado para proporcionar confidencialidad e integridad a los datos en redes no confiables) en el 2015.

RC4 por ser un algoritmo de cifrado de flujo genera una secuencia de bits pseudoaleatorios. Estos bits se combinan con el texto plano mediante la operación XOR para generar el texto cifrado. El proceso de descifrado se hace de la misma manera, ya que el XOR es una involución.

### **¿Cómo se genera la secuencia de bits pseudoaleatoria?**

Técnicamente, RC4 consta de dos algoritmos, un algoritmo de programación de clave (KSA, Key Scheduling Algorithm) y un algoritmo de generación pseudo-aleatoria (PRGA, Pseudo-Random Generation Algorithm). El KSA toma como entrada una clave K de longitud variable, típicamente entre 40 y 256 bits y produce el estado interno inicial st0 = (i, j, S), donde S es la representación canónica de una permutación del conjunto [0, 255] como un arreglo de bytes, e 'i, j' son ínidces en este arreglo. El PRGA, dado un estado interno str, produce el siguiente byte de la secuencia de clave Zr+1, junto con el estado interno actualizado str+1. Es importante destacar que el índice 'j' actualizado se calcula en función de los ínidces 'i, j y S' actuales, mientras que i es simplemente un contador (módulo 256).

<div align = "center">
    <img src = Images/Flowchart-of-RC4-Algorithm.png width = 35%>
    <p>Algoritmo de RC4</p>
</div>

### **Key Scheduling Algorithm (KSA)**
```pseudo
    for i from 0 to 255
        S[i] := i
    endfor
    j := 0
    for i from 0 to 255
        j := (j + S[i] + key[i mod keylenght]) mod 256
        swap values of S[i] and S[j]
    endfor
```
- El KSA se utiliza para inicializar la matriz 'S' en la permutación de identidad
- Posteriormente, se procesa 'S' durante 256 iteraciones de manera similar a la PRGA principal, pero también se mezclan bytes de la clave al mismo tiempo.
- 'keylenght' se define como el número de bytes en la clave y su rango es 1 ≤ keylength ≤ 256.

**Implementación en Python:**

In [25]:
#KSA
def KSA(k): # Input: key K of l bytes
    # Inicialización de la matriz 's' en la permutación identidad
    s = list(range(256))
    # Valor inicial del índice 'j'
    j = 0
    # Procesamiento de 's' durante 256 iteraciones y mezcla de bytes
    for i in range (0, 256):
        j = (j + s[i] + k[i % len(k)]) % 256
        s[i], s[j] = s[j], s[i] #swap
    return s # Output: estado interno st0

### **Pseudo-random Generation Algorithm (PRGA)**

```pseudo
   i := 0
   j := 0
   while GeneratingOutput:
      i := (i + 1) mod 256
      j := (j + S[i]) mod 256
      swap values of S[i] and S[j]
      K := S[(S[i] + S[j]) mod 256]
      output K
   endwhile
```
- El byte de salida se selecciona buscando los valores S(i) y S(j), sumándolos módulo 256, el resultado se utiliza como ínice en 'S' y el byte en esa posición será parte de la secuencia pseudoaleatoria.

**Implementación en Python**

In [26]:
#PRGA
def PRGA(s):
    # Valor inicial de los índices
    i = 0
    j = 0
    while True:
        i = (i + 1) % 256
        j = (j + s[i]) % 256
        s[i], s[j] = s[j], s[i] # swap
        K = s[(s[i] + s[j]) % 256]
        yield K # devuelve el valor de K y pausa temporalmente su ejecución

### **Test Vectors**

Los siguientes vectores de prueba se utilizarán para verificar la funcionalidad de la implementación de RC4. Las 'keys' y 'plaintext' están en ASCII, el 'keystream' y 'ciphertext' están en hexadecimal.

<div align = "center">
    <img src = Images/test-vectors.png></img>
</div>

#### **RC4 en Python**

In [27]:
def RC4_algorithm (key, plaintext):
    # Inicializamos el estado interno de 's'
    s = KSA(key)
    # Genera la secuencia pseudoaleatoria
    keystream = PRGA(s)
    # Ciframos el mensaje combinando cada byte del mensaje con el siguiente byte en la secuencia pseudoaleatoria
    ciphertext = bytes([p ^ next(keystream) for p in plaintext])
    return ciphertext.hex().upper()

- **Input Key** = Key
- **Plaintext** = Plaintext
- **Ciphertext** = BBF316E8D940AF0AD3

In [28]:
key1 = "Key".encode('utf=8')
text1 = "Plaintext".encode('utf-8')

assert RC4_algorithm(key1, text1) == 'BBF316E8D940AF0AD3'

print("El texto cifrado es el esperado", RC4_algorithm(key1, text1))

Pass! Resultado: BBF316E8D940AF0AD3
